As the name suggests, this notebook retrives the average edge travel time from all the SUMO simulations of different number of vehicles and travel times for analysis purpose. The outer loop for the number of vehicles and inner loops for green time and speed limit (to get the freeflow_travel_time). 

The edge_prob capability of SUMO is the source of the output and the edge travel time is averaged across every 60 seconds of the experiement excluding the first 60 seconds and when there is no traffic anymore. 

In [3]:
import xml.etree.ElementTree as ET
import csv
import os
import pandas as pd


length=992.80

In [ ]:
for numVeh in range(1000,1001,100):
    
    summary_all_travel_times=pd.DataFrame()
    
    #make new directory for output /output_mean_travel_time
    new_dir=f'./data/{numVeh}vehicles/output_mean_travel_time'
    os.makedirs(new_dir,exist_ok=True)
    
    
    for green_time in range(20,61,1): #20-60
        
        for speed_limit in range(10,51,1): #10-50

            freeflow_travel_time=length/speed_limit
        
            # Load and parse the XML file
            file_path=f'data/{numVeh}vehicles/green{green_time}s/freeflow{freeflow_travel_time}s/fcd_output.xml'
            # Parse the XML file
            tree = ET.parse(file_path)
            root = tree.getroot()
            
            # Initialize a dictionary to store the time spent on each lane by each vehicle
            vehicle_lane_times = {}
            
            # Iterate over each timestep in the XML
            for timestep in root.findall('timestep'):
                time = float(timestep.get('time'))
                for vehicle in timestep.findall('vehicle'):
                    vehicle_id = vehicle.get('id')
                    lane_id = vehicle.get('lane')
                    
                    if vehicle_id not in vehicle_lane_times:
                        vehicle_lane_times[vehicle_id] = {}
                    if lane_id not in vehicle_lane_times[vehicle_id]:
                        vehicle_lane_times[vehicle_id][lane_id] = 0
                    
                    vehicle_lane_times[vehicle_id][lane_id] += 1  # Assuming each timestep is 1 second
            
            # Convert the dictionary to a DataFrame
            data = []
            for vehicle_id, lanes in vehicle_lane_times.items():
                for lane_id, time_spent in lanes.items():
                    data.append([vehicle_id, lane_id, time_spent])
            
            df = pd.DataFrame(data, columns=['Vehicle_ID', 'Lane_ID', 'Time_Spent'])
    
            # Filter out rows where Lane_ID starts with ":"
            filtered_df = df[~df['Lane_ID'].str.startswith(':')]
            
            # Calculate the average time spent on each Lane_ID, averaged over the number of vehicles on the lanes
            average_time_travel = filtered_df.groupby('Lane_ID').apply(lambda x: x['Time_Spent'].sum() / x['Vehicle_ID'].nunique()).reset_index()
            
            # Rename the columns for clarity
            average_time_travel.columns = ['edge_id', 'average_traveltime']

            #adding the green time and freeflow_travel_time
            average_time_travel['green_time']=green_time
    
            average_time_travel['freeflow_travel_time']=freeflow_travel_time
    
            summary_all_travel_times=pd.concat([summary_all_travel_times,average_time_travel],ignore_index=True)
        
        
    
        
    
    print(f"number of vehicle  = {numVeh}, summary_all_travel_times")
    
    print(summary_all_travel_times)
    
    #save summary as csv with all travel times of different green times 
    summary_all_travel_times.to_csv(f'./data/{numVeh}vehicles/output_mean_travel_time/{numVeh}cars_summary_mean_travel_time.csv',index=False)